In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime

In [2]:
# Import the file as a Pandas DataFrame
fp = '../data/201904-citibike-tripdata.csv/201904-citibike-tripdata.csv'
nyc = pd.read_csv(fp)
nyc.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,88,2019-04-01 00:00:14.7410,2019-04-01 00:01:42.8900,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,36515,Subscriber,1982,1
1,443,2019-04-01 00:00:20.8270,2019-04-01 00:07:44.2920,254,W 11 St & 6 Ave,40.735324,-73.998004,540,Lexington Ave & E 29 St,40.743116,-73.982154,17406,Subscriber,1983,1
2,662,2019-04-01 00:00:30.9960,2019-04-01 00:11:33.2610,3244,University Pl & E 8 St,40.731437,-73.994903,3244,University Pl & E 8 St,40.731437,-73.994903,20775,Subscriber,1988,1
3,86,2019-04-01 00:00:30.9110,2019-04-01 00:01:56.9600,526,E 33 St & 5 Ave,40.747659,-73.984907,474,5 Ave & E 29 St,40.745168,-73.986831,36270,Subscriber,1982,1
4,136,2019-04-01 00:00:32.5930,2019-04-01 00:02:49.3910,461,E 20 St & 2 Ave,40.735877,-73.982050,504,1 Ave & E 16 St,40.732219,-73.981656,36051,Subscriber,1986,1


In [3]:
type(nyc.loc[0,'starttime'])

str

In [4]:
type(nyc.loc[0,'start station longitude'])

numpy.float64

In [5]:
# Setting the right format for starttime and stoptime
nyc['starttime'] = nyc['starttime'].str[:-5]
nyc['stoptime'] = nyc['stoptime'].str[:-5]
nyc['starttime'] = pd.to_datetime(nyc['starttime'])
nyc['stoptime'] = pd.to_datetime(nyc['stoptime'])


# Define the startime as index and create a new column
nyc = nyc.set_index('starttime')
nyc['type'] = 'station'
nyc.head(1)

,tripduration,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,type
starttime,,,,,,,,,,,,,,,
2019-04-01 00:00:14,88,2019-04-01 00:01:42,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,36515,Subscriber,1982,1,station


In [6]:
# Aggregate number of trips for each start station by hour of the day
start = nyc.pivot_table('tripduration', 
                     index = ['start station id', 
                              'start station latitude', 
                              'start station longitude', 
                              nyc.index.hour],
                     columns = 'type',
                     aggfunc='count').reset_index()


start.head()

type,start station id,start station latitude,start station longitude,starttime,station
0,72,40.767272,-73.993929,0,14
1,72,40.767272,-73.993929,1,9
2,72,40.767272,-73.993929,2,11
3,72,40.767272,-73.993929,3,11
4,72,40.767272,-73.993929,4,6


In [7]:
#To get a daily average we will divide by the numbers of days.
days = nyc.index.day.max()
start['station'] = start['station']/days

In [8]:
#Now, for comfort, we will change the name of the columns and define the color we want the points to have on the map.

# Rename the columns
start.columns = ['station_id', 'lat', 'lon', 'hour', 'count']

# Define the color
start['fillColor'] = '#53c688'

# The stops where less than one daily trip
# will have a different color
start.loc[start['count']<1, 'fillColor'] = '#586065'
start.head(1)

,station_id,lat,lon,hour,count,fillColor
0,72,40.767272,-73.993929,0,0.466667,#586065


In [9]:
def create_geojson_features(df):
    features = []
    
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['lon'],row['lat']]
            },
            'properties': {
                'time': pd.to_datetime(row['hour'], unit='h').__str__(),
                'style': {'color' : ''},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row['fillColor'],
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': row['count'] + 5
                }
            }
        }
        features.append(feature)
    return features


In [10]:
start_geojson = create_geojson_features(start)
start_geojson[0]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-73.99392888, 40.76727216]},
 'properties': {'time': '1970-01-01 00:00:00',
  'style': {'color': ''},
  'icon': 'circle',
  'iconstyle': {'fillColor': '#586065',
   'fillOpacity': 0.8,
   'stroke': 'true',
   'radius': 5.466666666666667}}}

In [11]:
# Create an empty list
df_hour_list = []

# Create a series with the different hours of the day
hours = pd.Series(nyc.index.hour.unique().sort_values())

# Create a list of points for each hour of the day
def create_list(hour):
    df_hour_list.append(nyc.loc[nyc.index.hour == hour,
                                ['start station latitude',
                                 'start station longitude']].
                        groupby(['start station latitude', 
                                 'start station longitude']).sum().reset_index().values.tolist())
hours.apply(create_list);

In [12]:
from folium.plugins import HeatMapWithTime


# Add trip events to the map
map_time = folium.Map(location=(40.76727216,-73.99392888), 
                      tiles="CartoDB Positron", 
                      zoom_start=12)


HeatMapWithTime(df_hour_list, 
                auto_play=True, 
                max_opacity=0.5, 
                gradient = {0.2: '#FBD973', 
                            0.4: '#fa782f', 
                            0.75: '#F16578', 
                            1: '#782890'}).add_to(map_time)
map_time.save('../maps/map_time.html')
map_time